# ETL 02: Health Facilities Data Preparation 

In this notebook, we will process the raw Excel data obtained from the IBB Open Data Portal.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
# Load the raw Excel file
excel_file = "saglik-tesisleri.xlsx"
df = pd.read_excel(excel_file, engine="openpyxl")

print("--- COLUMN NAMES ---")
print(df.columns.tolist())

print("\n--- SAMPLE DATA ---")
print(df.head(3))

--- COLUMN NAMES ---
['Sağlık Tesisi Adı', 'Ana Kategori', 'Alt Kategori', 'İlçe Adı', 'Mahalle Adı', 'ADRES', 'Latitude', 'Longitude']

--- SAMPLE DATA ---
                              Sağlık Tesisi Adı Ana Kategori  \
0           Ada Medikal Diğer Tıbbi Malz Satışı       Sağlık   
1  Uzman Dental Ağız ve Diş Sağlığı Polikliniği       Sağlık   
2                         Özel Moda Polikliniği       Sağlık   

                          Alt Kategori  İlçe Adı  Mahalle Adı  \
0                              Medikal    BEYKOZ      YALIKÖY   
1  Özel Ağız ve Diş Sağlığı Merkezleri  BAĞCILAR  YAVUZ SELİM   
2                      Poliklinik Özel   KADIKÖY     CAFERAĞA   

                                             ADRES   Latitude  Longitude  
0                Çayır Cad. No:25 F YALIKÖY/BEYKOZ  41.139505  29.086857  
1           Birlik Cad. No:15 YAVUZ SELİM/BAĞCILAR  41.035575  28.853850  
2  Mühürdar Karakolu Sk. No:1 -3A CAFERAĞA/KADIKÖY  40.984577  29.024374  


In [3]:
# When we inspected the sample data (df.head()) in the previous step, we noticed the first entry was 'Medikal' (Medical Supply Store), not a hospital.
# This suggests that the dataset includes all health-related facilities.
# So, we need to list all unique facility categories to decide which ones should stay.
df['Alt Kategori'].unique()

array(['Medikal', 'Özel Ağız ve Diş Sağlığı Merkezleri',
       'Poliklinik Özel', 'Doktor/Muayenehane', 'Gözlükçü/Optik',
       'Psikologlar', 'Diş Hekimi', 'Aile Sağlığı Merkezi', 'Veteriner',
       'Diyetisyen', 'Göz Merkezi Özel',
       'Psikoteknik Değerlendirme Merkezi',
       'İşitme Cihazı Satış ve Uygulama Merkezi', 'Kızılay/Kan Merkezi',
       'Ecza Deposu', 'Belediye Sağlık Merkezi', 'Devlet Hastanesi',
       'Tıp Merkezi Özel', 'Diş Laboratuvarı', 'Ağız Diş Sağlığı Merkezi',
       'Özel Hastane', 'Diyaliz Merkezi Özel', 'Sağlık Kabini Özel',
       'Laboratuvar Özel', 'Sağlık Diğer',
       'Fizik Tedavi ve Rehabilitasyon Merkezi Özel',
       'Verem Savaş Dispanseri', 'Tüp Bebek Merkezi', 'Diyaliz Merkezi',
       'Üniversite Hastanesi', 'Rehabilitasyon ve Aile Danışma Merkezi',
       'Kadın Doğum ve Çocuk Hastanesi', 'Semt Poliklinikleri',
       'Acil Yardım İstasyonu', 'Evde Bakım Merkezleri',
       'Yaşlı Bakım Evi/Huzurevi', 'Toplum Sağlığı Merkezi',
       '

In [4]:
# Define the list of core medical facilities to keep
target_categories = [
    'Devlet Hastanesi',
    'Özel Hastane',
    'Üniversite Hastanesi',
    'Eğitim Araştırma Hastanesi',
    'Şehir Hastanesi',
    'Kadın Doğum ve Çocuk Hastanesi',
    'Tıp Merkezi Özel'
]

df_hospitals = df[df['Alt Kategori'].isin(target_categories).copy()]

print(f"Original Data Count: {len(df)}")
print(f"Filtered Hospital Count: {len(df_hospitals)}")
print('-' * 30)
print(df_hospitals['Alt Kategori'].value_counts())

Original Data Count: 20469
Filtered Hospital Count: 690
------------------------------
Özel Hastane                      269
Tıp Merkezi Özel                  162
Devlet Hastanesi                   81
Şehir Hastanesi                    66
Eğitim Araştırma Hastanesi         56
Üniversite Hastanesi               45
Kadın Doğum ve Çocuk Hastanesi     11
Name: Alt Kategori, dtype: int64


In [5]:
# Define column names based on the output above
LAT_COL = 'Latitude'
LON_COL = 'Longitude'

# Remove rows with empty coordinates
df_clean = df_hospitals.dropna(subset=[LAT_COL, LON_COL]).copy()

# Create geometry: Point(x, y) => Point(Longitude, Latitude)
geometry = [Point(xy) for xy in zip(df_clean[LON_COL], df_clean[LAT_COL])]

# Create GeoDataFrame (EPSG:4326 for GPS coordinates)
gdf_hospitals = gpd.GeoDataFrame(df_clean, geometry=geometry, crs='EPSG:4326')

print(f"Total Hospitals: {len(gdf_hospitals)}")


Total Hospitals: 690


In [6]:
gdf_hospitals.head(10)

,Sağlık Tesisi Adı,Ana Kategori,Alt Kategori,İlçe Adı,Mahalle Adı,ADRES,Latitude,Longitude,geometry
126,Bayrampaşa Devlet Hastanesi,Sağlık,Devlet Hastanesi,BAYRAMPAŞA,İSMET PAŞA,Kenar Cad. No:6 İSMET PAŞA/BAYRAMPAŞA,41.041463,28.903141,POINT (28.90314 41.04146)
127,Özel Fizyodem Tıp Merkezi,Sağlık,Tıp Merkezi Özel,BAYRAMPAŞA,ORTA,Demirkapı Cad. No:47 -49 ORTA/BAYRAMPAŞA,41.037068,28.909220,POINT (28.90922 41.03707)
137,Medi World Tıp Merkezi,Sağlık,Tıp Merkezi Özel,BAYRAMPAŞA,YENİDOĞAN,Abdi İpekçi Cad. No:46 A YENİDOĞAN/BAYRAMPAŞA,41.037973,28.912395,POINT (28.9124 41.03797)
156,Özel Avrasya Hastanesi,Sağlık,Özel Hastane,ZEYTİNBURNU,BEŞTELSİZ,101. Sk. No:107 BEŞTELSİZ/ZEYTİNBURNU,41.003261,28.907044,POINT (28.90704 41.00326)
157,Özel Güney Hastanesi,Sağlık,Özel Hastane,ESENLER,MENDERES,Kazım Karabekir Cad. No:3 MENDERES/ESENLER,41.043810,28.879521,POINT (28.87952 41.04381)
161,Garanti Sağlık Merkezi,Sağlık,Tıp Merkezi Özel,BAKIRKÖY,CEVİZLİK,Hatboyu Çıkışı Sk. No:8 CEVİZLİK/BAKIRKÖY,40.980381,28.874543,POINT (28.87454 40.98038)
323,Uzunoğlu Özel Sağlık Hizmetleri,Sağlık,Tıp Merkezi Özel,KADIKÖY,RASİMPAŞA,Halitağa Cad. No:46 RASİMPAŞA/KADIKÖY,40.993028,29.030210,POINT (29.03021 40.99303)
412,Yeditepe Üniversitesi Koşuyolu Hastanesi,Sağlık,Üniversite Hastanesi,KADIKÖY,KOŞUYOLU,Koşuyolu Cad. No:168 KOŞUYOLU/KADIKÖY,41.004167,29.031112,POINT (29.03111 41.00417)
417,Özel Medi Time Cerrahi Tıp Merkezi,Sağlık,Tıp Merkezi Özel,ATAŞEHİR,FERHATPAŞA,Yeditepe Cad. No:85 FERHATPAŞA/ATAŞEHİR,40.986799,29.171436,POINT (29.17144 40.9868)
432,Çapa Hastahanesi Kadın Doğum Merkezi,Sağlık,Kadın Doğum ve Çocuk Hastanesi,FATİH,TOPKAPI,Turgut Özal Millet Cad. No:118 AC TOPKAPI/FATİH,41.016656,28.934296,POINT (28.9343 41.01666)


In [7]:
# Save to GeoJSON
output_file = "istanbul_hospitals.geojson"
gdf_hospitals.to_file(output_file, driver='GeoJSON')
print(f"Success! Data saved to '{output_file}'")

# Quick Map Check (Using Folium logic for visualization)
import folium

# Map center
m = folium.Map(location=[41.0082, 28.9784], zoom_start=10, tiles="CartoDB positron")

for idx, row in gdf_hospitals.iterrows():
    color = 'cadetblue'
    if 'Özel' in row['Alt Kategori']:
        color = 'orange'
    elif 'Devlet' in row['Alt Kategori']:
        color = 'darkblue'
    elif 'Üniversite' in row['Alt Kategori']:
        color = 'red'
        
    folium.CircleMarker(
        location = [row.geometry.y, row.geometry.x],
        radius = 3,
        color = color,
        fill = True,
        fill_opacity = 0.7,
        popup = f"{row['Alt Kategori']}:{row.get('Sağlık Tesisi Adı', 'Unknown')}"
    ).add_to(m)

m

Success! Data saved to 'istanbul_hospitals.geojson'
